In [10]:
from bs4 import BeautifulSoup, Tag
from dateutil.relativedelta import relativedelta
import requests
import datetime
from tqdm import tqdm
import pandas as pd


In [3]:
# Initialize the array with the starting date
from_date = datetime.datetime(2024, 5, 1)
dates = [from_date]

# Add one month at a time until we reach today
while dates[-1] < datetime.datetime.today():
    dates.append(dates[-1] + relativedelta(months=1))

In [ ]:
found_articles: list[Tag] = []

def met_condition(anchor) -> bool:
    lower = anchor.get_text().lower()
    return 'consumo' in lower and 'agua' in lower


for date in dates:
    base_page = f"https://bogota.gov.co/archivo/{date.strftime('%Y%m')}?tipo=article"
    html_doc = requests.get(base_page).content
    soup = BeautifulSoup(html_doc)
    anchors = soup.find_all("a", recursive=True)

    last_page_anchor = next((a for a in anchors if 'ltima' in a.get('title', '').lower()), None)

    if not last_page_anchor: break

    last_page = int(last_page_anchor.get('href').split('=')[-1])

    for i in tqdm(range(0, last_page)):
        url = f'{base_page}&page={i}'
        r = html_doc if html_doc and i == 0 else requests.get(url).content
        soup = BeautifulSoup(r)
        anchors = soup.find_all("a", recursive=True)

        filtered_anchors = [a for a in anchors if met_condition(a)]
        found_articles += filtered_anchors
    

100%|██████████| 31/31 [00:30<00:00,  1.01it/s]


In [12]:
urls = [f'https://bogota.gov.co/{a.get("href")}' for a in found_articles]
df_urls = pd.DataFrame(urls, columns=['URL'])
df_urls.to_csv('../data/embalces_urls.csv', index=False)